In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002871274948120117
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 17.511860874630003
200000 32.94276276590263
300000 33.179857820338206
400000 32.42488000602418
500000 32.38426984832083
600000 32.68583911050688
700000 33.8432391199642
800000 32.46353670809737
900000 32.06729331555271
1000000 32.1055175328437
1100000 31.583212265363503
1200000 31.953913500158194
1300000 15.646571108722437
1400000 23.815828061892432
1500000 31.67406784048615
1600000 31.567260250674384
1700000 31.54568509932811
1800000 31.992236065356682
1900000 32.27830016595121
2000000 33.85269768503415
2100000 32.150486086057796
2200000 31.917556378117524
2300000 32.35528114508848
2400000 32.89047041607821
2500000 32.547290207760625
2600000 12.90587422172192
2700000 28.096645914940346
2800000 40.69356542066249
2900000 41.73815217391668
3000000 40.7040324

29700000 28.701611969541254
29800000 31.130433596944826
29900000 31.920640249080066
30000000 32.686540581974576
30100000 32.526045778159855
30200000 31.855952481727698
30300000 33.1032725860601
30400000 34.61150595720777
30500000 33.123797930314666
30600000 32.148537540370796
30700000 12.651881771081737
30800000 34.6478870004398
30900000 33.93330543116039
31000000 33.17657058313061
31100000 20.404866087420363
31200000 24.311935800083866
31300000 32.81055443865696
31400000 32.92739320405853
31500000 32.93337946328507
31600000 32.57934265718309
31700000 33.357934833395404
31800000 32.30326351095655
31900000 30.99783941684387
32000000 31.295716981003807
32100000 30.31142566910593
32200000 7.428144655248657
32300000 31.29173709002317
32400000 30.6284140858179
32500000 30.849115281535262
32600000 32.1208197943983
32700000 32.42093075758555
32800000 31.08505878288026
32900000 32.457357015098424
33000000 32.314781243609765
33100000 32.575199155658474
33200000 20.41749869134275
33300000 25.559

59500000 30.016267187979288
59600000 15.664146092019553
59700000 23.549965772064937
59800000 29.606269870564685
59900000 28.18806544288015
60000000 28.469665600911718
60100000 27.49571940849135
60200000 26.85361951800885
60300000 26.905038212007536
60400000 22.888382908754274
60500000 11.73920920860461
60600000 26.592773277134082
60700000 27.33922136074982
60800000 26.851258176178387
60900000 27.77255675216478
61000000 28.390954659932017
61100000 28.217248268474723
61200000 28.793118766342477
61300000 13.511116434943029
61400000 27.478085942972182
61500000 29.56542292401754
61600000 29.142398524460653
61700000 30.33642994126481
61800000 29.3601900522117
61900000 29.304406662677973
62000000 29.506841661252828
62100000 29.921189406008917
62200000 13.571008632583295
62300000 25.599216506593926
62400000 28.232919404330865
62500000 28.017673409531834
62600000 28.495153925926388
62700000 27.24900532976675
62800000 26.487161779494794
62900000 26.23440440427341
63000000 16.04996832055774
63100

89300000 25.240397324117655
89400000 8.870333382545471
89500000 26.896159143324574
89600000 26.628618998810815
89700000 26.500738558067226
89800000 26.845178794727218
89900000 27.204100150535265
90000000 28.396033000842532
90100000 26.648228024646528
90200000 27.569800191068435
90300000 27.053935006052825
90400000 26.82749940101378
90500000 26.060394692322802
90600000 26.53745061743896
90700000 10.171050238836477
90800000 23.614197979029672
90900000 24.766897766380556
91000000 24.13671920465075
91100000 24.593481559527717
91200000 24.741480064437066
91300000 11.537806263518188
91400000 19.214252044570497
91500000 24.735946501343925
91600000 24.309889681495825
91700000 24.166967797247228
91800000 24.664898562509414
91900000 17.444449146080377
92000000 18.70990251845849
92100000 26.01505439232818
92200000 26.0084672643401
92300000 27.350817838327124
92400000 26.859800531325504
92500000 27.901492429805387
92600000 26.62034540442239
92700000 26.91689746136149
92800000 27.15643478141095
929

118400000 24.458909312420023
118500000 23.95925643310331
118600000 23.75631335319757
118700000 23.992491080021985
118800000 22.462342800892298
118900000 21.837915511278446
119000000 21.769240208282675
119100000 21.44368336594181
119200000 6.952782207953036
119300000 21.769940003297588
119400000 21.927534851894773
119500000 20.750769710932687
119600000 7.363855465098065
119700000 19.826511474217746
119800000 21.08521748507195
119900000 21.549638529821614
120000000 11.700689621016643
120100000 18.379184823982467
120200000 21.40734692811847
120300000 21.3618706762583
120400000 21.357361383663836
120500000 22.82433405345577
120600000 22.960183207451028
120700000 23.582360322848004
120800000 23.748074811029596
120900000 23.76203282071154
121000000 23.952176679706128
121100000 23.54276759070305
121200000 22.874503546409873
121300000 23.49011459882958
121400000 22.051948481946926
121500000 21.83748088450041
121600000 21.443050153920666
121700000 21.905013241852643
121800000 6.560296858624972


...

233000000 12.150118055294906
233100000 9.121130136998739
233200000 6.004719975477348
233300000 11.387273117219337
233400000 11.7130954260435
233500000 11.539379364422928
233600000 12.27404225556786
233700000 11.68879108774564
233800000 12.766572951262237
233900000 12.834705656613203
234000000 12.14572748236642
234100000 12.730827839953184
234200000 12.570100774262412
234300000 12.329696850600687
234400000 12.628468827378487
234500000 10.952454747911965
234600000 11.754972685337615
234700000 10.58720330620221
234800000 11.76009967964113
234900000 3.926442822791406
235000000 11.810868938344594
235100000 10.793192614435467
235200000 11.641188068987246
235300000 4.614651166661793
235400000 9.845548848812001
235500000 10.75940501629204
235600000 11.357549201112441
235700000 8.061461796396221
235800000 7.216727555051266
235900000 10.71121403937918
236000000 11.652265235367954
236100000 11.2479751833938
236200000 12.059617961502584
236300000 13.407529068707586
236400000 13.03146282809089
2365

261900000 9.392527320874184
262000000 9.389083089799076
262100000 9.51321936222996
262200000 10.055387428321916
262300000 8.992273805132573
262400000 9.652025973371298
262500000 9.317151417128231
262600000 9.376757106909858
262700000 9.227106139056303
262800000 8.919153239516026
262900000 7.781877258252911
263000000 4.6827652372494395
263100000 8.282921538363347
263200000 8.428041005809092
263300000 8.331708213830161
263400000 8.300452061182497
263500000 8.578064869509477
263600000 2.452083872635607
263700000 8.179690409943046
263800000 8.622629217102384
263900000 8.575375617697123
264000000 8.410507607578664
264100000 8.469386787485902
264200000 5.823612645970049
264300000 5.9090632115580854
264400000 9.082971298143297
264500000 9.081374834732502
264600000 9.115141225357533
264700000 9.246205025894744
264800000 9.52272953150367
264900000 9.84140796469858
265000000 9.656540236450912
265100000 9.18623462208036
265200000 9.515576774410706
265300000 9.05063370829345
265400000 8.9842027320

291200000 6.493024556300241
291300000 6.316563967001996
291400000 6.20744117879238
291500000 5.9636101588254995
291600000 5.992422165471836
291700000 5.980071167569995
291800000 6.029550282286461
291900000 1.1582773705169849
292000000 6.193078915521318
292100000 5.9768503810081715
292200000 5.915384236391038
292300000 5.892901256075003
292400000 6.21333544091121
292500000 6.2413222018276535
292600000 6.354183804533698
292700000 3.959874619539636
292800000 4.888893905964806
292900000 6.557857897516537
293000000 6.5997877267643545
293100000 6.347567501954872
293200000 6.729373174187091
293300000 6.759915358248987
293400000 6.3111000861738935
293500000 6.6839645317234275
293600000 2.1316889272940682
293700000 6.935754928976494
293800000 6.018013378021723
293900000 6.158876865114945
294000000 6.240085855383958
294100000 5.599431812302241
294200000 5.946299327050857
294300000 5.5422027322844745
294400000 4.439832518070315
294500000 2.846046569470924
294600000 5.650808138601081
294700000 5.6

320300000 3.294722824220252
320400000 3.382255337796307
320500000 3.393240189672162
320600000 3.4425740077436133
320700000 3.6129361928868398
320800000 3.418152492190987
320900000 3.5307784388197807
321000000 3.5414936896856766
321100000 3.533371406070781
321200000 1.935198001072522
321300000 2.7918851110484533
321400000 3.645288850640563
321500000 3.4436901010758767
321600000 3.2616978051763703
321700000 1.5150071175677122
321800000 3.469379997980927
321900000 3.4172326044794237
322000000 3.564716494182724
322100000 3.3658923761860677
322200000 3.2244996808556006
322300000 3.409612466138464
322400000 3.3272581029755735
322500000 3.2649688134448915
322600000 3.1637871896884024
322700000 2.0699951235644267
322800000 1.933212744692566
322900000 3.1158044147601527
323000000 3.268155593933346
323100000 3.2036456390255257
323200000 3.2385662613305013
323300000 3.17786571927797
323400000 3.162177635245661
323500000 3.309797462632233
323600000 3.208244219995949
323700000 3.2859841711483186
32

349000000 0.6054755153392315
349100000 0.5505415577317377
349200000 0.5047784791430626
349300000 0.488740920108372
349400000 0.5484955974021607
349500000 0.49870939323531455
349600000 0.4845802170345246
349700000 0.1755727919808315
349800000 0.2515295780543069
349900000 0.3644952642833988
350000000 0.3920516458612614
350100000 0.32019144233280683
350200000 0.3508502160740475
350300000 0.2965716141177257
350400000 0.30410560235610207
350500000 0.35137684888401627
350600000 0.27523703780280484
350700000 0.2545215265761389
350800000 0.24369543522770803
350900000 0.2786659626762622
351000000 0.14517818604023058
351100000 0.1874145989781221
351200000 0.2411463395240903
351300000 0.25572785453540486
351400000 0.20885229399340746
351500000 0.1649734721697854
351600000 0.22137476598196226
351700000 0.19762530265382727
351800000 0.16175100490929484
351900000 0.1871971021822731
352000000 0.13369483393868806
352100000 0.21832838822715417
352200000 0.21897900419945782
best so far: 0
type: [1, 1, 1